<a href="https://colab.research.google.com/github/josh-millar/josh-millar/blob/main/7001_2022_23_lab6_ABSA_with_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 6 - Aspect-Based Sentiment Analysis with BERT

In this lab we turn a pre-trained BERT model into a trainable Keras layer and apply it to the Aspect-Based Sentiment Analysis (ABSA). You can find the task description from (https://aclanthology.org/D19-1654.pdf).
This task provides a review text dataset with aspects.
Given a review and an aspect, we need to classify the sentiment conveyed towards this aspect on a three-point scale: POSITIVE, NEUTRAL or NEGATIVE.
This is a multi-class classification task.

BERT (Bidirectional Embedding Representations from Transformers) is a new model for pre-training language representations that obtains state-of-the-art results on many NLP tasks. We demonstrate how to integrate BERT as a custom Keras layer using the Huggingface library. 

In this lab, you will learn: 

1) How to use the Huggingface library.

2) How to integrate BERT in the models built previously. 

3) How to use the TPU from Colab. **Note**: Running BERT on the CPU will be very slow. Thus we recommend you to do this lab on a Colab TPU provided by Google.

In [1]:
import keras
import numpy as np
from keras.layers import Lambda, GlobalAveragePooling1D, Dense, Embedding
from keras import backend as K
from keras.models import Sequential
import matplotlib.pyplot as plt

from keras.layers import LSTM, RNN, Dropout, Input, LeakyReLU, Bidirectional,Conv1D, GlobalMaxPooling1D
from keras.layers.core import Dense
from keras.models import Model


We first need to install the Huggingface's Transformers package. You can find the relevant doc from [here](https://huggingface.co/transformers/index.html).

In [2]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 8.9 MB/s eta 0:00:00


## Preprocessing and Tokenization

In this lab we will use DistilBERT (https://arxiv.org/pdf/1910.01108.pdf) instead of BERT: DistilBERT is a smaller and faster Transformer model trained via distilling BERT. It has 40% less parameters than the bert-base-uncased model. It runs 60% faster, while preserving 97% of BERT’s performance as measured on the GLUE language understanding benchmark.

It is easy to switch between DistilBERT and BERT using the Huggingface transformers package. This package provides many pre-trained and pre-built models that are easy to use via a few lines of code.

Before using DistilBERT or BERT, we need a tokenizer. Generally speaking, every BERT related model has its own tokenizer, trained for this particular model. 
We can get the DistilBERT tokenizer from the **DistilBertTokenizer.from_pretrained** method.

In [3]:
from transformers import DistilBertTokenizer, RobertaTokenizer 
import tqdm
distil_bert = 'distilbert-base-uncased' # Pick a pre-trained model

# Defining DistilBERT tokenizer
tokenizer = DistilBertTokenizer.from_pretrained(distil_bert, do_lower_case=True, add_special_tokens=True,
                                                max_length=128, pad_to_max_length=True)

def tokenize(sentences, tokenizer, pad_length=128, pad_to_max_length=True ):
    if type(sentences) == str:
        inputs = tokenizer.encode_plus(sentences, add_special_tokens=True, max_length=pad_length, pad_to_max_length=pad_to_max_length, 
                                             return_attention_mask=True, return_token_type_ids=True)
        return np.asarray(inputs['input_ids'], dtype='int32'), np.asarray(inputs['attention_mask'], dtype='int32'), np.asarray(inputs['token_type_ids'], dtype='int32')
    
    input_ids, input_masks, input_segments = [],[],[]
    for sentence in sentences:
        inputs = tokenizer.encode_plus(sentence, add_special_tokens=True, max_length=pad_length, pad_to_max_length=pad_to_max_length, 
                                             return_attention_mask=True, return_token_type_ids=True)
        input_ids.append(inputs['input_ids'])
        input_masks.append(inputs['attention_mask'])
        input_segments.append(inputs['token_type_ids'])        
        
    return np.asarray(input_ids, dtype='int32'), np.asarray(input_masks, dtype='int32'), np.asarray(input_segments, dtype='int32')


Then we can use this tokenizer to tokenize our data. When working with word2vec and GloVe, we tokenized sentences into words ourselves and then converted the tokens into GloVe indices. But in BERT, we must use the BERT tokenizer that uses sub-word tokens.

For example, for the sentence: **This is a pretrained model.** our previous word-based tokenizer will generate the following tokens:

**"this", "is", "a", "pretrained", "model", "."**

Then you will find out that the token "pretrained" is not in the GloVe word dictionary. Thus we can not assign it a trained word vector.

The BERT tokenizer will separate the word "pretrained" into three sub-word tokens:

**'pre', '##train', '##ed'**

BERT thus uses these three token vectors to represent the word "pretrained". You will also see that the BERT tokenizer adds the special [CLS] token and the sentence separator [SEP] token.

In [4]:
inputs = tokenizer.tokenize("The capital of France is [MASK].")
print(inputs,'\n')

inputs = tokenizer.tokenize("This is a pretrained model.")
print(inputs,'\n')

ids,masks,segments = tokenize("The capital of France is [MASK].", tokenizer)
print(ids)
print(masks)
print(tokenizer.convert_ids_to_tokens(ids),"\n")

ids,masks,segments = tokenize("The capital of France is [MASK].", tokenizer, pad_to_max_length=False)
print(ids)
print(masks)
print(tokenizer.convert_ids_to_tokens(ids),"\n")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


['the', 'capital', 'of', 'france', 'is', '[MASK]', '.'] 

['this', 'is', 'a', 'pre', '##train', '##ed', 'model', '.'] 

[ 101 1996 3007 1997 2605 2003  103 1012  102    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
[1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


## Loading Data

### Downloading and preprocessing data

Unlike the IMDB dataset that is included and preprocessed by Keras, the dataset we will be using is the aspect-based sentiment analysis (ABSA)  dataset, which consists of 5,297 labeled reviews. These are split into 4,297 reviews for training and 500 reviews for testing and validation, respectively. 

For ABSA, sentiment polarities were assigned with respect to the aspect terms.  The start and end positions for each aspect term are provided.


In [5]:
import requests
def downloadfile(url):
  rq = requests.get(url)
  open(url.split('/')[-1], 'wb').write(rq.content)
downloadfile('https://raw.githubusercontent.com/siat-nlp/MAMS-for-ABSA/master/data/MAMS-ATSA/raw/train.xml')
downloadfile('https://raw.githubusercontent.com/siat-nlp/MAMS-for-ABSA/master/data/MAMS-ATSA/raw/val.xml')
downloadfile('https://raw.githubusercontent.com/siat-nlp/MAMS-for-ABSA/master/data/MAMS-ATSA/raw/test.xml')


# The code is modified from https://raw.githubusercontent.com/siat-nlp/MAMS-for-ABSA/master/data_process/utils.py
from xml.etree.ElementTree import parse

def parse_sentence_term(path, lowercase=False):
    tree = parse(path)
    sentences = tree.getroot()
    data = []
    split_char = '__split__'
    for sentence in sentences:
        text = sentence.find('text')
        if text is None:
            continue
        text = text.text
        if lowercase:
            text = text.lower()
        aspectTerms = sentence.find('aspectTerms')
        if aspectTerms is None:
            continue
        for aspectTerm in aspectTerms:
            term = aspectTerm.get('term')
            if lowercase:
                term = term.lower()
            polarity = aspectTerm.get('polarity')
            start = aspectTerm.get('from')
            end = aspectTerm.get('to')
            piece = [text , term,  polarity , start , end]
            data.append(piece)
    return data
train = parse_sentence_term("train.xml",True)
dev = parse_sentence_term("val.xml",True)
test = parse_sentence_term("test.xml",True)

print("Training entries: {}".format(len(train)))
print("Test entries: {}".format(len(test)))


Training entries: 11186
Test entries: 1336


Let’s first see some examples from the data:

In [6]:
print("SENTENCE \t ASPECT \t LABEL \t ASPECT-START-INDEX \t ASPECT-END-INDEX")
print([a[1] for a in train[0:5]])
# print(train[1])
# print(train[2])
# print(train[3])
# print(train[4])

SENTENCE 	 ASPECT 	 LABEL 	 ASPECT-START-INDEX 	 ASPECT-END-INDEX
['decor', 'food', 'prices', 'tables', 'manager']


Using the BERT `tokenize` function above, we can text and aspect words to integers:


In [7]:
# Please write your code to generate the following data
x_train_review_int, x_train_review_masks, _ = tokenize([r[0] for r in train], tokenizer)
x_train_aspect_int, x_train_aspect_masks, _ = tokenize([r[1] for r in train], tokenizer)

x_dev_review_int, x_dev_review_masks, _ = tokenize([r[0] for r in dev], tokenizer)
x_dev_aspect_int, x_dev_aspect_masks, _ = tokenize([r[1] for r in dev], tokenizer)

x_test_review_int, x_test_review_masks, _ = tokenize([r[0] for r in test], tokenizer)
x_test_aspect_int, x_test_aspect_masks, _ = tokenize([r[1] for r in test], tokenizer)
# x_train_review_int
# x_train_review_masks
# x_train_aspect_int
# x_train_aspect_masks

# x_dev_review_int
# x_dev_review_masks
# x_dev_aspect_int
# x_dev_aspect_masks

# x_test_review_int
# x_test_review_masks
# x_test_aspect_int
# x_test_aspect_masks

# your code goes here

# You can check the results as follows:
assert len(x_train_aspect_int) == len(train)
assert len(x_train_aspect_masks) == len(x_train_aspect_int)
assert len(x_test_aspect_int) == len(test)
assert len(x_test_aspect_masks) == len(x_test_aspect_int)
print("x_dev_aspect_int[0]:")
print(x_dev_aspect_int[0])
print("x_dev_aspect_masks[0]:")
print(x_dev_aspect_masks[0])
print("x_dev_review_int[0]:")
print(x_dev_review_int[0])
print("x_dev_review_masks[0]:")
print(x_dev_review_masks[0])

x_dev_aspect_int[0]:
[ 101 8974  102    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
x_dev_aspect_masks[0]:
[1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
x_dev_review_int[0]:
[  101  2044  1037  323

We one-hot encode the labels, using 4 (Binary:100) to represent "positive", 2 (Binary:010) for "neutral", and 1 (Binary:001) for "negative".

In [8]:
def label2int(dataset):
  y = []
  for example in dataset:
    if example[2].lower() == "negative":
      y.append([0,0,1])
    elif example[2].lower() == "neutral":
      y.append([0,1,0])
    else:
      # assert example[2].lower() == "positive"
      y.append([1,0,0])
  return y
  
y_train = label2int(train)
y_dev = label2int(dev)
y_test = label2int(test)
y_train = np.array(y_train)
y_dev = np.array(y_dev)
y_test = np.array(y_test)

print(y_train[0])
print(y_train[1])
print(y_train[2])
print(y_train[3])
print(y_train[4])

[0 0 1]
[1 0 0]
[1 0 0]
[0 1 0]
[0 0 1]


Now we have almost finished the data preprocessing. Unlike in the previous labs, there are two model inputs: review and aspect. The easiest way is to combine the review and aspect into one sentence and then input it into the model.

In [14]:
# Please write your code to combine sentences and aspect words into the following varibles

# x_train_int
# x_train_masks
# x_dev_int
# x_dev_masks
# x_test_int
# x_test_masks

# Tips: 
# 1) Use the special token <SEP> to concatenate sentences and aspect words
# 2) Make sure they are paded/truncated to a max length

# your code goes here

x_train_sentences = [r[0] for r in train]
x_train_aspects = [r[1] for r in train]
x_train_joined = []
for idx, sentence in enumerate(x_train_sentences):
    x_train_joined.append(' [SEP] '.join([sentence, x_train_aspects[idx]]))
x_train_int, x_train_masks, _ = tokenize(x_train_joined, tokenizer)

x_dev_sentences = [r[0] for r in dev]
x_dev_aspects = [r[1] for r in dev]
x_dev_joined = []
for idx, sentence in enumerate(x_dev_sentences):
    x_dev_joined.append(' [SEP] '.join([sentence, x_dev_aspects[idx]]))
x_dev_int, x_dev_masks, _ = tokenize(x_dev_joined, tokenizer)

x_test_sentences = [r[0] for r in test]
x_test_aspects = [r[1] for r in test]
x_test_joined = []
for idx, sentence in enumerate(x_test_sentences):
    x_test_joined.append(' [SEP] '.join([sentence, x_test_aspects[idx]]))
x_test_int, x_test_masks, _ = tokenize(x_test_joined, tokenizer)

# Don't forget the to use the np.array function to wrap the outputs
x_train_int_np = np.array(x_train_int)
x_train_masks_np = np.array(x_train_masks)
x_dev_int_np = np.array(x_dev_int)
x_dev_masks_np = np.array(x_dev_masks)
x_test_int_np = np.array(x_test_int)
x_test_masks_np = np.array(x_test_masks)


print(x_dev_int[0])
print(x_dev_masks[0],'\n')
print(x_dev_int_np[0])
print(x_dev_masks_np[0]) # sentence + aspect


[  101  2044  1037  3232  1997  8974  1010  1996 18726  1011  1011  1045
  2066  1996 27940  1013 24792  2621  4897  1998  1996 13675 11514  6508
 26852  1011  1011  2175  2091  2307  1012   102  8974   102     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

## Model 1: Prebuilt Sequence Classification



The Huggingface transformer package provides many prebuilt models. Now let us try a sequence classification model based on DistilBERT. 

The models with BERT are much bigger than our previous models. To run it faster, we can use TPU. Detailed guidelines on how to use TPU can be found from https://www.tensorflow.org/guide/tpu.

In [24]:
from transformers import TFDistilBertForSequenceClassification, DistilBertConfig
import tensorflow as tf

distil_bert = 'distilbert-base-uncased'

config = DistilBertConfig(num_labels=3)
config.output_hidden_states = False

def create_TFDistilBertForSequenceClassification():
  transformer_model = TFDistilBertForSequenceClassification.from_pretrained(distil_bert, config = config)
  input_ids = tf.keras.layers.Input(shape=(128,), name='input_token', dtype='int32')
  input_masks_ids = tf.keras.layers.Input(shape=(128,), name='masked_token', dtype='int32')
  X = transformer_model(input_ids, input_masks_ids)
  print(X)
  return tf.keras.Model(inputs=[input_ids, input_masks_ids], outputs = X)

use_tpu = True
if use_tpu:
  # Create distribution strategy
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  strategy = tf.distribute.TPUStrategy(tpu)

  # Create model on TPU:
  with strategy.scope():
    model = create_TFDistilBertForSequenceClassification()
    optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
else:
  model = create_TFDistilBertForSequenceClassification()
  model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_transform', 'vocab_projector', 'activation_13', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'pre_classifier', 'dropout_96']
You should probably TRAIN this model on a down-stream task to be able to use i

TFSequenceClassifierOutput(loss=None, logits=<KerasTensor: shape=(None, 3) dtype=float32 (created by layer 'tf_distil_bert_for_sequence_classification_1')>, hidden_states=None, attentions=None)


In [16]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_token (InputLayer)       [(None, 128)]        0           []                               
                                                                                                  
 masked_token (InputLayer)      [(None, 128)]        0           []                               
                                                                                                  
 tf_distil_bert_for_sequence_cl  TFSequenceClassifie  66955779   ['input_token[0][0]',            
 assification (TFDistilBertForS  rOutput(loss=None,               'masked_token[0][0]']           
 equenceClassification)         logits=(None, 3),                                                 
                                 hidden_states=None                                           

In [17]:
history = model.fit([x_train_int_np,x_train_masks_np],
                    y_train,
                    epochs=30,
                    batch_size=512,
                    validation_data=([x_dev_int_np,x_dev_masks_np], y_dev),
                    verbose=1)

Epoch 1/30
22/22 [==============================] - 103s 2s/step - loss: 0.8053 - accuracy: 0.4279 - val_loss: 0.5899 - val_accuracy: 0.4677
Epoch 2/30
22/22 [==============================] - 5s 224ms/step - loss: 0.5787 - accuracy: 0.4891 - val_loss: 0.5743 - val_accuracy: 0.5150
Epoch 3/30
22/22 [==============================] - 5s 230ms/step - loss: 0.5210 - accuracy: 0.6156 - val_loss: 0.4638 - val_accuracy: 0.7072
Epoch 4/30
22/22 [==============================] - 5s 224ms/step - loss: 0.4305 - accuracy: 0.7354 - val_loss: 0.4378 - val_accuracy: 0.7417
Epoch 5/30
22/22 [==============================] - 5s 223ms/step - loss: 0.4178 - accuracy: 0.7424 - val_loss: 0.4846 - val_accuracy: 0.7635
Epoch 6/30
22/22 [==============================] - 5s 230ms/step - loss: 0.3581 - accuracy: 0.8019 - val_loss: 0.4752 - val_accuracy: 0.7740
Epoch 7/30
22/22 [==============================] - 5s 223ms/step - loss: 0.3414 - accuracy: 0.8158 - val_loss: 0.4747 - val_accuracy: 0.7875
Epoch 8

In [18]:
results = model.evaluate([x_test_int_np,x_test_masks_np], y_test)
print(results)

42/42 [==============================] - 8s 100ms/step - loss: 0.7821 - accuracy: 0.7964
[0.7820532321929932, 0.796407163143158]



## Model 2: Neural bag of words using BERT

Here we will use model2 from Lab3 with BERT instead of the previous static word embeddings.

In [19]:
class GlobalAveragePooling1DMasked(GlobalAveragePooling1D):
    def call(self, x, mask=None):
        if mask != None:
            return K.sum(x, axis=1) / K.sum(mask, axis=1)
        else:
            return super().call(x)

In [20]:
from transformers import TFDistilBertModel, DistilBertConfig

def get_BERT_layer():
  distil_bert = 'distilbert-base-uncased'
  config = DistilBertConfig(dropout=0.2, attention_dropout=0.2)
  config.output_hidden_states = False
  return TFDistilBertModel.from_pretrained(distil_bert, config = config)


In [28]:
import tensorflow as tf
hdepth=16
MAX_SEQUENCE_LENGTH = 128
EMBED_SIZE=100


def create_bag_of_words_BERT():
  # your code goes here
  input_ids_in = tf.keras.layers.Input(shape=(128,), name='input_token', dtype='int32')
  input_masks_in = tf.keras.layers.Input(shape=(128,), name='masked_token', dtype='int32')
  bert_model = get_BERT_layer()
  bert_layer = bert_model(input_ids_in, input_masks_in)
  averaging_layer = GlobalAveragePooling1DMasked()(bert_layer.last_hidden_state)
  hidden_layer = Dense(16)(averaging_layer)
  label = Dense(3, activation='sigmoid')(hidden_layer)
  return Model(inputs=[input_ids_in,input_masks_in], outputs=[label],name='Model2_BERT')

use_tpu = True
if use_tpu:
  # Create distribution strategy
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  strategy = tf.distribute.TPUStrategy(tpu)

  # Create model
  with strategy.scope():
    model2 = create_bag_of_words_BERT()
    optimizer2 = tf.keras.optimizers.Adam(learning_rate=5e-5)
    model2.compile(optimizer=optimizer2, loss='binary_crossentropy', metrics=['accuracy'])
else:
  model2 = create_bag_of_words_BERT()
  model2.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_transform', 'vocab_projector', 'activation_13', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [29]:
model2.summary() 

Model: "Model2_BERT"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_token (InputLayer)       [(None, 128)]        0           []                               
                                                                                                  
 masked_token (InputLayer)      [(None, 128)]        0           []                               
                                                                                                  
 tf_distil_bert_model_6 (TFDist  TFBaseModelOutput(l  66362880   ['input_token[0][0]',            
 ilBertModel)                   ast_hidden_state=(N               'masked_token[0][0]']           
                                one, 128, 768),                                                   
                                 hidden_states=None                                     

In [30]:

history = model2.fit([x_train_int_np,x_train_masks_np],
                    y_train,
                    epochs=30,
                    batch_size=512,
                    validation_data=([x_dev_int_np,x_dev_masks_np], y_dev),
                    verbose=1)

Epoch 1/30
22/22 [==============================] - 105s 2s/step - loss: 0.6135 - accuracy: 0.4601 - val_loss: 0.5551 - val_accuracy: 0.5270
Epoch 2/30
22/22 [==============================] - 5s 232ms/step - loss: 0.4988 - accuracy: 0.6068 - val_loss: 0.4192 - val_accuracy: 0.7237
Epoch 3/30
22/22 [==============================] - 5s 222ms/step - loss: 0.3607 - accuracy: 0.7520 - val_loss: 0.3312 - val_accuracy: 0.7890
Epoch 4/30
22/22 [==============================] - 5s 226ms/step - loss: 0.2876 - accuracy: 0.8152 - val_loss: 0.3068 - val_accuracy: 0.8078
Epoch 5/30
22/22 [==============================] - 5s 232ms/step - loss: 0.2327 - accuracy: 0.8567 - val_loss: 0.3124 - val_accuracy: 0.8153
Epoch 6/30
22/22 [==============================] - 5s 222ms/step - loss: 0.1881 - accuracy: 0.8918 - val_loss: 0.3344 - val_accuracy: 0.8251
Epoch 7/30
22/22 [==============================] - 5s 224ms/step - loss: 0.1522 - accuracy: 0.9093 - val_loss: 0.3568 - val_accuracy: 0.8153
Epoch 8

In [31]:
results = model2.evaluate([x_test_int_np,x_test_masks_np], y_test)
print(results)

42/42 [==============================] - 8s 90ms/step - loss: 0.6486 - accuracy: 0.8301
[0.6485996246337891, 0.830089807510376]


## Model 3: CNN or LSTM with BERT

Please follow the architecture for model2 to construct a CNN or an LSTM model on the top of BERT. 

In [35]:
def create_bag_of_words_BERT():
  # your code goes here
  input_ids_in = tf.keras.layers.Input(shape=(128,), name='input_token', dtype='int32')
  input_masks_in = tf.keras.layers.Input(shape=(128,), name='masked_token', dtype='int32')
  bert_model = get_BERT_layer()
  bert_layer = bert_model(input_ids_in, input_masks_in)
  lstm_layer = LSTM(100, return_sequences=False)(bert_layer.last_hidden_state)
  label = Dense(3, activation='sigmoid')(lstm_layer)
  return Model(inputs=[input_ids_in,input_masks_in], outputs=[label],name='Model2_BERT')

use_tpu = True
if use_tpu:
  # Create distribution strategy
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  strategy = tf.distribute.TPUStrategy(tpu)

  # Create model
  with strategy.scope():
    model3 = create_bag_of_words_BERT()
    optimizer3 = tf.keras.optimizers.Adam(learning_rate=5e-5)
    model3.compile(optimizer=optimizer3, loss='binary_crossentropy', metrics=['accuracy'])
else:
  model3 = create_bag_of_words_BERT()
  model3.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_transform', 'vocab_projector', 'activation_13', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [36]:
model3.summary() 

Model: "Model2_BERT"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_token (InputLayer)       [(None, 128)]        0           []                               
                                                                                                  
 masked_token (InputLayer)      [(None, 128)]        0           []                               
                                                                                                  
 tf_distil_bert_model_8 (TFDist  TFBaseModelOutput(l  66362880   ['input_token[0][0]',            
 ilBertModel)                   ast_hidden_state=(N               'masked_token[0][0]']           
                                one, 128, 768),                                                   
                                 hidden_states=None                                     

In [37]:

history = model3.fit([x_train_int_np,x_train_masks_np],
                    y_train,
                    epochs=30,
                    batch_size=512,
                    validation_data=([x_dev_int_np,x_dev_masks_np], y_dev),
                    verbose=1)

Epoch 1/30
22/22 [==============================] - 112s 3s/step - loss: 0.5982 - accuracy: 0.4706 - val_loss: 0.5499 - val_accuracy: 0.5255
Epoch 2/30
22/22 [==============================] - 7s 299ms/step - loss: 0.4945 - accuracy: 0.6168 - val_loss: 0.4347 - val_accuracy: 0.6982
Epoch 3/30
22/22 [==============================] - 6s 293ms/step - loss: 0.3853 - accuracy: 0.7328 - val_loss: 0.3442 - val_accuracy: 0.7643
Epoch 4/30
22/22 [==============================] - 6s 295ms/step - loss: 0.3030 - accuracy: 0.8042 - val_loss: 0.3127 - val_accuracy: 0.8033
Epoch 5/30
22/22 [==============================] - 6s 294ms/step - loss: 0.2552 - accuracy: 0.8426 - val_loss: 0.3056 - val_accuracy: 0.8048
Epoch 6/30
22/22 [==============================] - 6s 292ms/step - loss: 0.2168 - accuracy: 0.8728 - val_loss: 0.3142 - val_accuracy: 0.8093
Epoch 7/30
22/22 [==============================] - 7s 297ms/step - loss: 0.1774 - accuracy: 0.8993 - val_loss: 0.3159 - val_accuracy: 0.8153
Epoch 8

In [38]:
results = model3.evaluate([x_test_int_np,x_test_masks_np], y_test)
print(results)

42/42 [==============================] - 9s 110ms/step - loss: 0.5127 - accuracy: 0.8234
[0.5126556754112244, 0.8233532905578613]
